## Solve regional power flow with MLP load predictions 

1. Define path to Master file corresponding to TGW year and scenario
2. Solve PF
3. Save lines and Xfer data
4. Optional: print network performance data

## Initialize

### Import packages

In [2]:
## Import packages
import time
import datetime
import os
import re
import json
import yaml
import joblib
import numpy as np 
import pandas as pd # to create data frames
import pyarrow as pa
import xarray as xr
import random
import matplotlib.pyplot as plt
import shutil #To enable duplicating files
from opendssdirect import dss
from src import physics_ops
from src import file_ops
from src import opendss_ops
from src import TGW_ops
from src import input_ops
from src import df_ops

### Load config file with scenarios and parameters 

In [8]:
config_file_name = 'opendss_config1'; config_path = f"/home/aviadf/opendss/notebooks/config/{config_file_name}.yaml"; config = input_ops.load_config(config_path)

enable_save = 1  # 1 enable | 0 disable

TGW_years_scenarios = config['TGW_years_scenarios']

demand_mode = config['demand_mode']
    
aggregation_level = config['aggregation_level']

CITY_REGIONS_TO_RUN = config['CITY_REGIONS_TO_RUN']

input_data_dict_name = config['input_data_dict_name']
aggregation_level = config['aggregation_level']
smart_ds_year = config['smart_ds_years'][0]
building_types = config["building_types"]

input_data_training_path = config['input_data_training_path']
CITY_REGIONS_TO_RUN = config['CITY_REGIONS_TO_RUN']
start_month = config['start_month']
end_month = config['end_month']

## Initialize parameters for saving paths
Y_column = config['Y_column']
input_data_prediction_path = config['input_data_prediction_path']
output_path_prediction_str = config['output_data_prediction_path']
output_pf_path = config['output_pf_path']

smart_ds_year = config['smart_ds_years'][0]
smart_ds_load_path = config['smart_ds_load_path'] + f"/{smart_ds_year}" # path to procesed smart-ds resstock data 
   
solution_mode = config['solution_mode']

## Define variables to create list of mdh to run
start_month_mdh = config['start_month_mdh'] 
end_month_mdh = config['end_month_mdh']
top_percent_mdh = config['top_percent_mdh']


## Load dictionary, sort by total city aggregated buildings demand, extract mdh of top % load hours
## load demand data 
regional_demand_weather_ampacity_path = smart_ds_load_path + f"/all_cities/aggregated_demand"
regional_demand_weather_ampacity_all_cities = joblib.load(os.path.join(regional_demand_weather_ampacity_path, "regional_demand_weather_ampacity_all_cities"))
## Sort your dictionary by aggregated total demand  
regional_demand_weather_ampacity_all_cities_sorted = df_ops.sort_nested_dict_dfs(regional_demand_weather_ampacity_all_cities, "aggregated_predicted_buildings_total_kw", ascending=False)
top_n_hours = int(np.ceil(8760*top_percent_mdh/100)) # calculate top city demand hours to run (top_percent_mdh% of hours of the year)
## Define start and end load hours to run
start_row_percent = config['start_row_percent']
start_row_idx = int(np.ceil(8760*start_row_percent/100)) # index from which to start loop of load hours
# start_row_idx = 0 # index from which to start loop of load hours
end_row_idx = top_n_hours 

# print(f"solution_mode: {solution_mode} \nCITY_REGIONS_TO_RUN: {CITY_REGIONS_TO_RUN} \n TGW year:{TGW_weather_year} \n TGW_scenario: {TGW_scenario} \n demand mode: {demand_mode}")
print(f"enable_save:{enable_save} \nsolution_mode: {solution_mode} \nTGW_years_scenarios:{TGW_years_scenarios} \nCITY_REGIONS_TO_RUN: {CITY_REGIONS_TO_RUN}  \n demand mode: {demand_mode}")

print(f"\n\ntop_percent_mdh: {top_percent_mdh}%, top_n_hours: {top_n_hours}, start_row_idx:{start_row_idx} ({start_row_percent}%), end_row_idx:{end_row_idx} ({top_percent_mdh}%) \n")

enable_save:1 
solution_mode: snapshot 
TGW_years_scenarios:{'2018': ['historical'], '2058': ['rcp45hotter']} 
CITY_REGIONS_TO_RUN: {'GSO': ['rural', 'industrial', 'urban-suburban'], 'AUS': ['P1R', 'P1U', 'P2U'], 'SFO': ['P1U', 'P2U', 'P1R']}  
 demand mode: MLP


top_percent_mdh: 5%, top_n_hours: 438, start_row_idx:88 (1%), end_row_idx:438 (5%) 



## Run power flow 
Note: took 40min for all scenarios and 3 TGW year-scenario combinations

In [1]:
# Auxilary parameter 
pf_converged = 0 # defult value 0,  0 did not converge | 1 converged

master_file_option = 'TGW' # Options: TGW | default of smart-ds

# list_of_mdh = [(6,19,13)]  # list of mdh to run, e.g., GSO rural regional total kw peak

start_time = time.time()

for TGW_weather_year, TGW_scenarios in TGW_years_scenarios.items():
    for TGW_scenario in TGW_scenarios:
        print(f"--- Starting scenario: {TGW_weather_year} {TGW_scenario} ---\n")
        year = TGW_weather_year; 
        # --- Iterate over all selected regions --- 
        for city, regions in CITY_REGIONS_TO_RUN.items():
            
            ## create list of mdh for top % hours
            df_city = regional_demand_weather_ampacity_all_cities_sorted[(TGW_weather_year, TGW_scenario)][city]   # load regional demand data to later create list of mdh for top % hours  
            list_of_mdh = df_ops.get_top_n_mdh(df_city, top_n_hours, start_month_mdh, end_month_mdh)

            for region in regions:
                print(f"--- Starting scenario: {city} {region} ---\n")
                # --- Iterate over all selected month-day-hour (mdh) --- 
                for row_i in range(start_row_idx, end_row_idx):
                    mdh = list_of_mdh[row_i]; m,d,h = mdh
                    print(f"--- Starting Index: {row_i}, mdh: {mdh} ---\n")


                    # --- Initialize region's paths ---
                    # Create path to regional master file 
                    if solution_mode == 'snapshot':
                        region_path = f"/nfs/turbo/seas-mtcraig-climate/SMART-DS/v1.0/{smart_ds_year}/{city}/{region}/scenarios/base_timeseries/opendss_no_loadshapes"
                    else:
                        region_path = f"/nfs/turbo/seas-mtcraig-climate/SMART-DS/v1.0/{smart_ds_year}/{city}/{region}/scenarios/base_timeseries/opendss"  

                    # --- Redirect opendss to new master file  ---
                    # redirect_start_time = time.time()
                    dss.Basic.ClearAll() 

                    if master_file_option == 'TGW':
                        # --- Redirect opendss to TGW scenario master file  ---
                        new_master_dir = os.path.join(region_path, "predicted_master_files", TGW_scenario, TGW_weather_year)
                        dss.Command(f'Redirect "{new_master_dir}/Master_{TGW_scenario}_{TGW_weather_year}_{m}_{d}_{h}.dss"') # Direct opendss engine to master file 
                    else:
                        # --- Redirect opendss to default master file  ---
                        dss.Command(f'Redirect "{region_path}/Master.dss"') # Direct opendss engine to master file 

                    # redirect_end_time = time.time(); print(f"Compiled Master_{TGW_scenario}_{TGW_weather_year}_{m}_{d}_{h} in ", (redirect_end_time - redirect_start_time) / 60, "minutes")

                    # --- Solve power flow ---
                    pf_start_time = time.time()
                    dss.Solution.Solve()  # Solve the circuit
                    if dss.Solution.Converged():
                        print(f"---Power flow solution converged successfully after {dss.Solution.Iterations()} iterations---\n")
                        pf_converged = 1
                    else:
                        print("---Power flow solution did not converge!!!!!!!!!!!!!!!!!---\n")
                        pf_converged = 0
                    pf_end_time = time.time(); print("PF solve runtime:", (pf_end_time - pf_start_time) / 60, "minutes")

                    # --- Extract line and transformers data (physical properties, loading, voltage etc.) ---
                    dict_bus_coord = opendss_ops.create_dict_bus_coord(region_path)  # Create a dictionary of bus names and coordinates from Buscoords.dss
                    line_df = opendss_ops.extract_line_information(dict_bus_coord, year,m,d,h,row_i) # Extract line data
                    transformer_df = opendss_ops.extract_transformer_information(dict_bus_coord, year,m,d,h,row_i) # Extract transformers data

                    # Add current year line and xfer data to multi-year data frame
                    if row_i == start_row_idx:
                        all_line_df = line_df
                        all_transformer_df = transformer_df
                    else:
                        # concat current df to a multi-year data frame
                        all_line_df = pd.concat([all_line_df, line_df], ignore_index=True)
                        all_transformer_df = pd.concat([all_transformer_df, transformer_df], ignore_index=True) 

                # --- Save results per region---     
                if master_file_option == 'TGW':
                    lines_file_name = f"lines_top_{start_row_percent}_{top_percent_mdh}_percent"
                    transformers_file_name = f"transformers_top_{start_row_percent}_{top_percent_mdh}_percent"
                    metadata_file_name = f"metadata_top_{start_row_percent}_{top_percent_mdh}_percent"
                else:
                    lines_file_name = f"lines_regional_peak_default"
                    transformers_file_name = f"transformers_regional_peak_default"
                    metadata_file_name = f"metadata_regional_peak_default"
                    
                # Initialize dictionaries
                lines_dict = {}
                transformers_dict = {}
                # Initialize inner dictionary per TGW scenario
                lines_dict[(TGW_weather_year,TGW_scenario)] = {}
                transformers_dict[(TGW_weather_year,TGW_scenario)] = {}
                # Add data frames to 2 level key (smart-ds) of dictionaries
                lines_dict[(TGW_weather_year,TGW_scenario)][(smart_ds_year, city, region)] = all_line_df
                transformers_dict[(TGW_weather_year,TGW_scenario)][(smart_ds_year, city, region)] = all_transformer_df
                
                if enable_save == 1:
                    # Save dictionaries as joblib files
                    if master_file_option == 'TGW':
                        predictions_dir = os.path.join(output_pf_path, f"{city}/{region}/{TGW_scenario}/{TGW_weather_year}/")
                    else:
                        predictions_dir = os.path.join(output_pf_path, f"{city}/{region}/smart_ds_default/{smart_ds_year}")

                    os.makedirs(predictions_dir, exist_ok=True) # Create directories if they don't exist
                    
                    if pf_converged == 1: 
                        joblib.dump(lines_dict, os.path.join(predictions_dir, f"{lines_file_name}.joblib"))
                        joblib.dump(transformers_dict, os.path.join(predictions_dir, f"{transformers_file_name}.joblib"))
                    else:
                        joblib.dump(lines_dict, os.path.join(predictions_dir, f"{lines_file_name}_did_not_converge.joblib"))
                        joblib.dump(transformers_dict, os.path.join(predictions_dir, f"{transformers_file_name}_did_not_converge.joblib"))

                    # Define metadata #### !!! Add time array and temp array
                    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                    metadata = {
                        "changes": "demand_only",
                        "solution_mode": solution_mode,
                        "smart_ds_year": smart_ds_year,
                        "timestamp": timestamp,
                    }
                    # Save metadata to a JSON file
                    metadata_file = f"{predictions_dir}{metadata_file_name}.json"
                    with open(metadata_file, 'w') as f:
                        json.dump(metadata, f)
                    print(f"---- Results for scenario {city} {region} saved succesfuly in {predictions_dir} ----\n")
    
end_time = time.time(); print("Total runtime:", (end_time - start_time) / 60, "minutes")

### Print bus voltages 

In [21]:
buses = dss.Circuit.AllBusNames()
for bus in buses:
    dss.Circuit.SetActiveBus(bus)
    voltages = dss.Bus.puVmagAngle()
    voltages_pu = voltages[::2]
    for v in voltages_pu:
        if v < 0.9 or v > 1.1:
            print(f"Voltage warning at {bus}: {v:.3f} pu")

Voltage warning at p1rdt1577lv: 0.889 pu
Voltage warning at p1rdt1577lv: 0.889 pu
Voltage warning at p1rdt2214: 0.890 pu
Voltage warning at p1rdt2214: 0.880 pu
Voltage warning at p1rdt2214: 0.881 pu
Voltage warning at p1rdt2214lv: 0.880 pu
Voltage warning at p1rdt2214lv: 0.881 pu
Voltage warning at p1rdt2215: 0.890 pu
Voltage warning at p1rdt2215: 0.880 pu
Voltage warning at p1rdt2215: 0.881 pu
Voltage warning at p1rdt2215lv: 0.877 pu
Voltage warning at p1rdt2215lv: 0.877 pu
Voltage warning at p1rdt2216: 0.889 pu
Voltage warning at p1rdt2216: 0.880 pu
Voltage warning at p1rdt2216: 0.882 pu
Voltage warning at p1rdt2216lv: 0.877 pu
Voltage warning at p1rdt2216lv: 0.877 pu
Voltage warning at p1rdt2217: 0.879 pu
Voltage warning at p1rdt2217lv: 0.872 pu
Voltage warning at p1rdt2217lv: 0.873 pu
Voltage warning at p1rdt2218: 0.879 pu
Voltage warning at p1rdt2218lv: 0.872 pu
Voltage warning at p1rdt2218lv: 0.873 pu
Voltage warning at p1rdt2219: 0.889 pu
Voltage warning at p1rdt2219: 0.879 pu
V

### Print circuit loads

In [5]:
# Get total losses
losses = dss.Circuit.Losses()
total_loss_kw = losses[0] / 1000  # Convert watts to kW          
# Get total circuit power 
total_circuit_power = dss.Circuit.TotalPower() #  sum of all the powers in the first terminal of each source (Vsource and Isource)
total_circuit_power_kw = - total_circuit_power[0]  # Real power in kW          
# Total load
total_circuit_Loads = total_circuit_power_kw -total_loss_kw

print(f'Total circuit load: {total_circuit_Loads} kw')

Total circuit load: 50400.27587415851 kw


### Print line and transformer data

In [6]:
# print("Line Information:")
# display(all_line_df.head(10))
# print("\nTransformer Information:")
# display(all_transformer_df.head(10))

top_10_loading = all_line_df.nlargest(10, 'Loading [%]')
display(top_10_loading)
top_10_loading = all_transformer_df.nlargest(10, 'Loading [%]')
display(top_10_loading)

,Weather year,Line,LineCode,Length [km],From_Lat,To_Lat,From_Long,To_Long,NormAmps [A],Nominal V [kV],Voltage [pu],Loading [%]
14821,0,breaker(r:rhs0_69-rts0_69)r_9274,breaker_3_3,0.001000,36.198024,36.199176,-79.613022,-79.612539,115.08,39.837169,0.990561,180.554722
11205,0,l(r:rdm3431-rdt2033lv),1p_oh_al_2/0_rucina_2,0.017970,36.189869,36.189866,-79.565413,-79.565618,115.08,0.120089,0.990575,157.073771
13715,0,l(r:rdt2712-rdt2716),3p_ug_al_3,0.076106,36.171453,36.171554,-79.701693,-79.701149,295.19,7.199558,1.017741,155.399856
14350,0,padswitch(r:rdt2712-rdt2716)r_9239,padswitch_3,0.001000,36.171494,36.171453,-79.701722,-79.701693,295.19,7.199558,1.017826,155.394473
14349,0,padswitch(r:rdt2716-rdt2728)r_9238,padswitch_3,0.001000,36.171607,36.171651,-79.701059,-79.701051,295.19,7.199558,1.015524,154.338375
13714,0,l(r:rdt2716-rdt2728),3p_ug_al_3,0.009974,36.171626,36.171607,-79.701185,-79.701059,295.19,7.199558,1.015759,154.338375
9247,0,recloser_disswitch(r:rdt1548-rdt1926)r_9925,recloser_3,0.001000,36.198899,36.198926,-79.629506,-79.629550,115.08,7.199558,0.969136,154.279732
14345,0,padswitch(r:rdt2717-rdt2718)r_9234,padswitch_3,0.001000,36.172059,36.172078,-79.700425,-79.700477,295.19,7.199558,1.013540,153.944229
13712,0,l(r:rdt2717-rdt2718),3p_ug_al_3,0.039816,36.171855,36.172059,-79.700752,-79.700425,295.19,7.199558,1.014489,153.944229
14344,0,padswitch(r:rdt2717-rdt2718)r_9233,padswitch_3,0.001000,36.171838,36.171855,-79.700804,-79.700752,295.19,7.199558,1.014572,153.941422


,Weather year,Transformer,Bus,Lat,Long,Num windings,Num phases,KV rating [kV],kVA rating [kVA],Loading [%],Voltages [kV]
2943,0,tr(r:rdt2616-rdt2616lv),rdt2616,36.175936,-79.690517,3,1,"[7.2, 0.12, 0.12]",9.25,226.600961,"[7.1396364942587525, 0.08461940365907238, 0.0,..."
413,0,tr(r:rdt442-rdt442lv),rdt442,36.259067,-79.595789,3,1,"[7.2, 0.12, 0.12]",9.25,217.807108,"[7.089585215904905, -0.15570370205626458, 0.0,..."
2397,0,tr(r:rdt2278-rdt2278lv),rdt2278,36.155852,-79.531618,3,1,"[7.2, 0.12, 0.12]",9.25,209.478191,"[6.822762811562537, -0.036643114091011356, 0.0..."
2655,0,tr(r:rdt891-rdt891lv),rdt891,36.220909,-79.536084,3,1,"[7.2, 0.12, 0.12]",9.25,202.776830,"[7.116687161580037, -0.034769035396885445, 0.0..."
1855,0,tr(r:rdt1674-rdt1674lv),rdt1674,36.204917,-79.650466,3,1,"[7.2, 0.12, 0.12]",23.12,188.554189,"[6.860811437767339, 0.0834496995405292, 0.0, 0..."
851,0,tr(r:rdt846-rdt846lv),rdt846,36.229314,-79.598585,3,1,"[7.2, 0.12, 0.12]",23.12,184.540425,"[6.886479930482205, -0.15654948795329768, 0.0,..."
2789,0,tr(r:rdt1247-rdt1247lv),rdt1247,36.186446,-79.532400,3,1,"[7.2, 0.12, 0.12]",23.12,182.439490,"[7.317021490757943, -0.15367456904004675, 0.0,..."
463,0,tr(r:rdt492-rdt492lv),rdt492,36.263092,-79.553846,3,1,"[7.2, 0.12, 0.12]",9.25,181.842075,"[7.377151103643927, -0.03864896602420429, 0.0,..."
472,0,tr(r:rdt501-rdt501lv),rdt501,36.254993,-79.561753,3,1,"[7.2, 0.12, 0.12]",9.25,181.403519,"[7.484212661384709, 0.08218369067746516, 0.0, ..."
3029,0,tr(r:rdt2702-rdt2702lv),rdt2702,36.170523,-79.705730,3,1,"[7.2, 0.12, 0.12]",9.25,179.925642,"[7.374260321095433, 0.0851211243355416, 0.0, 0..."


## Load saved PF data

In [17]:
# --- Save results ---     
lines_file_name = f"lines_regional_peak"
transformers_file_name = f"transformers_regional_peak"
metadata_file_name = f"metadata_regional_peak"

city = 'GSO'
region = 'rural'

TGW_weather_year = '2018'
TGW_scenario = 'historical'
predictions_dir = os.path.join(output_pf_path, f"{city}/{region}/{TGW_scenario}/{TGW_weather_year}/")
lines_dict_1 = joblib.load(os.path.join(predictions_dir, f"{lines_file_name}.joblib"))
transformers_dict_1 = joblib.load(os.path.join(predictions_dir, f"{transformers_file_name}.joblib"))

TGW_weather_year = '2058'
TGW_scenario = 'rcp45hotter'
predictions_dir = os.path.join(output_pf_path, f"{city}/{region}/{TGW_scenario}/{TGW_weather_year}/")
lines_dict_2 = joblib.load(os.path.join(predictions_dir, f"{lines_file_name}.joblib"))
transformers_dict_2 = joblib.load(os.path.join(predictions_dir, f"{transformers_file_name}.joblib"))

# TGW_weather_year = '2058'
# TGW_scenario = 'rcp45cooler'
# predictions_dir = os.path.join(output_pf_path, f"{city}/{region}/{TGW_scenario}/{TGW_weather_year}/")
# lines_dict_3 = joblib.load(os.path.join(predictions_dir, f"{lines_file_name}.joblib"))
# transformers_dict_3 = joblib.load(os.path.join(predictions_dir, f"{transformers_file_name}.joblib"))

lines_dict = lines_dict_1
transformers_dict = transformers_dict_1
lines_dict.update(lines_dict_2)
transformers_dict.update(transformers_dict_2)
# lines_dict.update(lines_dict_3)
# transformers_dict.update(transformers_dict_3)

In [11]:
lines_dict.keys()

dict_keys([('2018', 'historical'), ('2058', 'rcp45hotter')])

In [10]:
lines_dict[('2018', 'historical')].keys()

dict_keys([('2018', 'GSO', 'rural')])

In [18]:
lines_dict[('2018', 'historical')][('2018', 'GSO', 'rural')].head(2)

,Weather year,Line,LineCode,Length [km],From_Lat,To_Lat,From_Long,To_Long,NormAmps [A],Nominal V [kV],Voltage [pu],Loading [%]
0,0,sb1_rhs1_1247_35,3p_oh_code_3,0.001512,36.269344,36.269244,-79.611706,-79.611329,480.0,7.199558,1.024671,104.818707
1,0,sb1_rhs1_1247_33,breaker_13kv_3,0.001000,36.269576,36.269481,-79.611155,-79.611157,1200.0,7.199558,1.024851,41.927482
